In [3]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [4]:
import pandas as pd
import requests
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import process_data
from script import geolocaliser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# 2. Création du fichier des transactions cotières

On télécharge la base Demande de valeurs foncières (DVF) qui référence, pour l'année 2023, l'ensemble des mutations à titre onéreux (en majeure partie géolocalisées) : https://files.data.gouv.fr/geo-dvf/latest/csv/2023/full.csv.gz

Les informations sont issues de la Base nationale des données patrimoniales, alimentées par le système d'information de la DGFip et couvrent la France métropolitaine à l'exception des départements du Bas-Rhin, du Haut-Rhin et de Moselle.

La base recense des actes (id_mutation), qui comportent un ou plusieurs mutations distinctes, repérées par le numéro de disposition (numero_disposition).

Les observations de la base, appelées "lignes de restitution", concernent les différents locaux d'une mutation (Appartement, Maison, Dépendance, Local Industriel), ventilées selon autant de natures de culture présentes dans l'immeuble.

In [5]:
# url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20241008-071041/valeursfoncieres-2023.txt.zip"

url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/full.csv.gz"
# Envoyer une requête HTTP pour obtenir le fichier CSV

downloaded_file = "full.csv.gz"
response = requests.get(url)

with open(downloaded_file, 'wb') as file:
    file.write(response.content)

# Décompresser le fichier
with gzip.open(downloaded_file, 'rb') as f_in:
    with open("full.csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv("full.csv",encoding="utf-8")

# Optionnel : supprimer le fichier compressé après décompression
os.remove("full.csv.gz")
os.remove("full.csv")

C:\Users\flori\AppData\Local\Temp\ipykernel_2136\190562132.py:18: DtypeWarning: Columns (8,10,12,14,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv",encoding="utf-8")


In [6]:
## Première opération : alléger le dataset

colonnes_a_supprimer = ['adresse_suffixe',
                        'code_nature_culture',
                        'ancien_code_commune',
                        'ancien_nom_commune',
                        'ancien_id_parcelle',
                        'numero_volume',
                        'code_nature_culture_speciale',
                        'nature_culture_speciale',
                        'lot1_numero',
                        'lot2_numero',
                        'lot3_numero',
                        'lot4_numero',
                        'lot5_numero',
                        'lot1_surface_carrez',
                        'lot2_surface_carrez',
                        'lot3_surface_carrez',
                        'lot4_surface_carrez',
                        'lot5_surface_carrez'
                        ]

df.drop(columns=colonnes_a_supprimer, inplace=True)

On importe le fichier des communes cotières et on ne retient que les transactions liées à ces communes

In [7]:
# Effectuer un left join entre df_final et df_communes
df_cotieres = pd.read_csv('data/communes_cotieres.csv',sep=";")
df_cotieres = df_cotieres.rename(columns={'latitude': 'latitude_centre'})
df_cotieres = df_cotieres.rename(columns={'longitude': 'longitude_centre'})

liste_cotieres = sorted(df_cotieres['nom'].unique().tolist())

Les données présentes dans le fichier sont les suivantes :

Identifiant de mutation / Numéro de disposition: Chaque couple est un identifiant unique d'un acte de vente

Nature de la mutation: Il s'agit du type de vente qui a eu lieu. Il peut s'agir d'une vente classique, d'une vente en l’état futur d’achèvement, d'une vente de terrain à bâtir, d'une adjudication, ou d'une expropriation ou échange.

Valeur foncière: Montant de la vente. Il est TTC et n'inclut pas les frais de notaire et les éventuels frais d'agence.

Adresse: L'adresse exacte du bien est communiquée via plusieurs colonnes comme le numéro de voie, le code postal etc.

Latitude/Longitude : Remplies de manière presque exhaustive.

Informations cadastrales: Des informations cadastrales sont fournies telles que l'identifiant de parcelle.

Nombre de lots : Nombre de lots (restitués jusqu'à 5)

Type de local: Il peut s'agir d'une maison, d'un appartement, d'une dépendance (isolée), ou d'un local industriel et commercial ou assimilés.

Surface réelle bâti: Il s'agit de la surface réelle mesurée au sol entre les murs, différente de la surface Carrez.

Nombre de pièces principales du bien immobilier

Nature culture: Pour les terrains une nature de culture est renseignée afin de connaître son utilisation. Les types de terrains possible sont : terrains a bâtir, terrains d'agrément, bois, futaies feuillues, futaies mixtes, oseraies, peupleraies, futaies résineuses, taillis sous futaie, taillis simples, carrières, chemin de fer, eaux, jardins, landes, landes boisées, prés, pâtures, pacages, prés d'embouche, herbages, prés plantes, sols, terres, terres plantées, vergers, vignes

Surface Terrain: Surface cadastrale du terrain.

1er filtre : On restreint DVF aux seules communes côtières.

In [8]:
df = df[df['nom_commune'].isin(liste_cotieres)]

2nd filtre : on ne retient que les ventes.

In [9]:
df = df[df['nature_mutation']=='Vente']
df = df.drop(columns=['nature_mutation'])

3ème filtre : On ne retient que les mutations dont la valeur foncière est renseignée

In [10]:
df = df[df['valeur_fonciere']>0]

In [11]:
print(df.shape[0])
print(df['id_mutation'].unique().shape[0])

412848
179496


In [12]:
from IPython.display import display

# Fonction pour afficher le tableau filtré dans un joli format pour Jupyter Notebook
def afficher_tableau_par_id_mutation(df, id_mutation_str):
    # Filtrer le DataFrame en fonction de l'id_mutation
    df_filtered = df[df['id_mutation'].astype(str) == id_mutation_str]
    
    # Sélectionner les colonnes nécessaires
    df_filtered = df_filtered[['id_mutation', 'valeur_fonciere', 'type_local', 'surface_reelle_bati', 'surface_terrain','nature_culture', 'nombre_pieces_principales']]
    
    # Affichage du tableau joli avec pandas pour Jupyter Notebook
    display(df_filtered)


Cas le plus typique : un seul local dans une seule mutation

In [13]:
# Exemple d'utilisation
afficher_tableau_par_id_mutation(df, '2023-457573')

,id_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nature_culture,nombre_pieces_principales
1330716,2023-457573,85600.0,Appartement,55.0,NaN,NaN,3.0


#Cas n°2 : Plusieurs locaux dans une mutation. Dans ce cas, il faut identifier si la mutation concerne une maison, un appartement, ou autre chose.

In [14]:
afficher_tableau_par_id_mutation(df, '2023-457574')

,id_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nature_culture,nombre_pieces_principales
1330717,2023-457574,230000.0,Appartement,22.0,NaN,NaN,1.0
1330718,2023-457574,230000.0,Dépendance,NaN,NaN,NaN,0.0


Cas n°3 : Quand une disposition comporte plusieurs locaux ou plusieurs natures de culture, le fichier de restitution comporte autant de lignes qu’il y a de locaux ou de nature de culture concernés par la mutation.
Ainsi, pour une même publication, il peut y avoir 1 à n ligne(s) de restitution. Les données génériques (ainsi que le prix) sont alors répétées sur chaque ligne.
On retire d abord les nature_culture autre que "sols" puis on repère le type de local principal

In [15]:
afficher_tableau_par_id_mutation(df, '2023-457957')

,id_mutation,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nature_culture,nombre_pieces_principales
1331555,2023-457957,539000.0,Dépendance,NaN,280.0,terrains d'agrément,0.0
1331556,2023-457957,539000.0,Maison,111.0,280.0,terrains d'agrément,4.0
1331557,2023-457957,539000.0,Maison,111.0,750.0,sols,4.0
1331558,2023-457957,539000.0,Dépendance,NaN,750.0,sols,0.0


On ne retient que les surfaces d'habitation, donc celles avec nature_culture vide ou égale à sols.

In [16]:
df = df[df['nature_culture'].isna() | (df['nature_culture'] == 'sols')]


In [17]:
def process_group(group):
    
    # Filtrer les lignes où 'nature_culture' est différent de vide ou de "sols"
    
    maison_present = 'Maison' in group['type_local'].values
    appart_present = 'Appartement' in group['type_local'].values
    dependance_present = 'Dépendance' in group['type_local'].values
    
    bati_group = group[group['type_local'].isin(['Maison', 'Appartement'])]
    surface_reelle_bati = bati_group['surface_reelle_bati'].fillna(0).sum()
    surface_terrain = group['surface_terrain'].fillna(0).sum()
    
    # Vérifier les colonnes avant d'accéder à leur première valeur
    valeur_fonciere = group['valeur_fonciere'].iloc[0] if not group['valeur_fonciere'].empty else None
    nombre_pieces_principales = group['nombre_pieces_principales'].max() if not group['nombre_pieces_principales'].isna().all() else None
    id_mutation = group['id_mutation'].iloc[0]
    numero_dispositon = group['numero_disposition'].iloc[0]
    # Résultats
    result = {
        'id_mutation' : id_mutation,
        'numero_disposition' : numero_dispositon,
        'valeur_fonciere': valeur_fonciere,
        'surface_reelle_bati': surface_reelle_bati,
        'surface_terrain': surface_terrain,
        'nombre_locaux': len(group),
        'maison_present': maison_present,
        'appart_present': appart_present,
        'dependance': dependance_present,
        'nombre_pieces_principales': nombre_pieces_principales,
    }

    # Ajouter les colonnes supplémentaires (avec vérification)
    for col in group.columns:
        if col not in result:
            result[col] = group[col].iloc[0] if not group[col].empty else None
    
    return pd.Series(result)

# Appliquer le traitement avec le groupement par plusieurs colonnes
df = df.groupby(['id_mutation', 'numero_disposition']).apply(process_group)

# Réinitialiser l'index
df.reset_index(drop=True, inplace=True)

C:\Users\flori\AppData\Local\Temp\ipykernel_2136\2045021250.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['id_mutation', 'numero_disposition']).apply(process_group)


On ne retient que les appartements et les maisons

In [18]:
df = df[(df['appart_present'] == True) | (df['maison_present'] == True)]

# Assigner "Appartement" ou "Maison" à la colonne type_local
df['type_local'] = df.apply(
    lambda row: 'Maison' if row['maison_present'] else 'Appartement', axis=1
)

# Supprimer les colonnes appart_present et maison_present
df.drop(columns=['appart_present', 'maison_present'], inplace=True)

In [19]:
df = df[df['surface_reelle_bati'] > 0]
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

Il reste des doublons. La plupart concernent les exactes mêmes transactions (mais ont des numéros de disposition différentes). Seules quelques unes concernent des dispositions à des adresses différentes. Comme on ne peut pas différencier les prix, on les supprime

In [20]:
df.to_csv('save.csv',encoding='utf8',sep=';')

In [21]:
df = pd.read_csv('save.csv',encoding="utf8",sep=";")

C:\Users\flori\AppData\Local\Temp\ipykernel_2136\817348212.py:1: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('save.csv',encoding="utf8",sep=";")


In [22]:
# Une partie des transactions sont en doublon
df = df[~df.duplicated(subset=['id_mutation', 'prix_m2'], keep='first')]

df = df[~df.duplicated(subset=['id_mutation'], keep=False)]


In [ ]:
colonnes_a_nettoyer = ['adresse_numero', 'code_postal']
df = process_data.nettoyer_colonnes(df, colonnes_a_nettoyer)
# Convertir la colonne 'code_commune' en type string
df['code_commune'] = df['code_commune'].astype('string')

# Ajouter un '0' au début si la chaîne a 4 caractères
df['code_commune'] = [x.zfill(5) if len(x) == 4 else x for x in df['code_commune']]

# Vérifier les résultats
print(df['code_commune'].head())

float64
object
float64
object
0    76540
1    76540
2    76157
3    76103
4    76540
Name: code_commune, dtype: object


In [24]:
# 4ème opération (si besoin de géolocalisation) : Compléter par le type de voie

voie = pd.read_csv("data/voie.csv",sep=";",encoding="utf-8")
print(voie.head())

# Liste des abréviations de types de voie
abbreviations = voie['abreviation'].tolist()

# Appliquer la fonction à la colonne 'adresse_nom_voie'
result = [process_data.check_abbreviation(adresse,abbreviations) for adresse in df['adresse_nom_voie']]

# Décomposer les résultats dans les colonnes 'type_voie' et 'nom_voie'
df['type_voie'] = [x[0] for x in result]
df['nom_voie'] = [x[1] for x in result]

df = df.merge(voie,left_on=['type_voie'],right_on=['abreviation'])

  abreviation type_voie_complet
0         RUE               Rue
1          AV            Avenue
2         RTE             Route
3         CHE            Chemin
4          BD         Boulevard


In [25]:
## 5ème : Réécriture de l'adresse
df['Adresse'] = df['adresse_numero'] + ' ' + df['type_voie_complet'] + ' ' + df['nom_voie'] + ' ' + df['code_postal'] + ' ' + df['nom_commune']

display(df['Adresse'].head())

0           48 Rue DE CONSTANTINE 76000 Rouen
1           45 Rue DES CHARRETTES 76000 Rouen
2    14 Boulevard CLAUDE MONET 76380 Canteleu
3        20 Allée DES FLEURS 76240 Bonsecours
4                   35 Rue MUSTEL 76000 Rouen
Name: Adresse, dtype: string


In [26]:
# Charger les données des codes de région
region = pd.read_csv('data/code_region.csv', sep=';')

int64
object
   departmentCode           departmentName  regionCode  \
0               1                      Ain        84.0   
1               2                    Aisne        32.0   
2               3                   Allier        84.0   
3               4  Alpes-de-Haute-Provence        93.0   
4               5             Hautes-Alpes        93.0   

                   regionName  
0        Auvergne-Rhône-Alpes  
1             Hauts-de-France  
2        Auvergne-Rhône-Alpes  
3  Provence-Alpes-Côte d'Azur  
4  Provence-Alpes-Côte d'Azur  
Type de la colonne 'departmentCode': int64
Modalités de 'departmentCode':
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  21  22  23  24  25  26  27  28  29  20  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  9

In [28]:
#Fusionner df_geolocalisees avec les codes de région
df = df.merge(region, left_on=["code_departement"], right_on=['departmentCode'], how='left')

df['regionCode'] = df['regionCode'].astype('Int64')  # Utilise Int64 pour gérer les valeurs manquantes
#Compter le nombre de transactions par région

transactions_par_region = df.groupby('regionName').size()

# Calculer le total des transactions
total_transactions = transactions_par_region.sum()

# Calculer la part de chaque région dans le total des transactions
part_region = (transactions_par_region / total_transactions) * 100

# Créer un DataFrame avec les résultats
tableau_regions = pd.DataFrame({
    'Nombre de transactions': transactions_par_region,
    'Part du total (%)': part_region
})

# Afficher le tableau des régions
display(tableau_regions)

                            Nombre de transactions  Part du total (%)
regionName                                                           
Auvergne-Rhône-Alpes                           121           0.097716
Bourgogne-Franche-Comté                         71           0.057338
Bretagne                                     19146          15.461770
Centre-Val de Loire                             57           0.046032
Grand Est                                       42           0.033918
Guadeloupe                                       6           0.004845
Hauts-de-France                               6059           4.893077
Normandie                                    13988          11.296314
Nouvelle-Aquitaine                           19804          15.993152
Occitanie                                    17042          13.762638
Pays de la Loire                             13541          10.935330
Provence-Alpes-Côte d'Azur                   33857          27.341958
Île-de-France       

3. Calcul de la base des prix au mètre carré

Notre objectif est d'évaluer le prix au mètre carré des biens en distinguant les maisons et les appartements
Une mutation peut contenir plusieurs types de locaux

On conserve chaque couple mutation/disposition qui contient une nature de culture sols ou vide. On suppose que ces mutations concernent des biens à visée d'habitation. Pour ces mutations, on regarde si elles contiennent :
- un local de type 'Maison' :  la transaction est référencée comme une maison
- un local de type 'Appartement' (mais sans 'Maison') : la transaction est référencée comme un appartement
- aucun local de type 'Maison' ou 'Appartement' : la transaction n'est pas retenue

La valeur foncière étant dupliquée, seule sa première occurence est retenue. Le nombre de pièces retenu est le plus grand de toutes les lignes de restitution. Les surfaces réelles sont additionnées par mutation.

On vérifie que toutes les transactions concernent des surfaces non nulles

In [29]:
df_cotieres = df_cotieres.drop_duplicates(subset=['nom'])

In [30]:
df = pd.merge(df, df_cotieres, how='left', left_on='nom_commune', right_on='nom')

In [31]:
# Calcul de la moyenne des prix totale en fonction de la surface
prix_moyen_m2 = df['valeur_fonciere'].sum() / df['surface_reelle_bati'].sum()
print(f"Moyenne totale des prix en fonction de la surface : {prix_moyen_m2:.2f} €")

# Groupement par région et type_local pour calculer les prix moyens
prix_moyen = (
    df.groupby(['regionName', 'type_local'])
    .apply(lambda group: group['valeur_fonciere'].sum() / group['surface_reelle_bati'].sum())
    .reset_index(name='prix_moyen')
)

# Conversion en tableau croisé dynamique (pivot table)
tableau_prix_moyen = prix_moyen.pivot_table(
    index='regionName',
    columns='type_local',
    values='prix_moyen',
    aggfunc='mean'  # Non nécessaire ici car chaque cellule est déjà agrégée
)

# Remplir les valeurs manquantes par 0 ou autre
tableau_prix_moyen = tableau_prix_moyen.fillna(0)

# Calcul du nombre de transactions par région
nb_transactions_region = df.groupby('regionName').size()

# Calcul du pourcentage d'appartements et de maisons par région
nb_transactions_appart = df[df['type_local'] == "Appartement"].groupby('regionName').size()
nb_transactions_maison = df[df['type_local'] == "Maison"].groupby('regionName').size()

# Calculer le pourcentage pour chaque type par région
pourcentage_appart = (nb_transactions_appart / nb_transactions_region * 100).fillna(0)
pourcentage_maison = (nb_transactions_maison / nb_transactions_region * 100).fillna(0)


Moyenne totale des prix en fonction de la surface : 3901.89 €


C:\Users\flori\AppData\Local\Temp\ipykernel_2136\1610620510.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group['valeur_fonciere'].sum() / group['surface_reelle_bati'].sum())


In [33]:
# Création de la variable adresse_py en remplaçant les espaces par des "+"
df['adresse_py'] = df['Adresse'].str.replace(' ', '+', regex=True)

In [34]:
commune = pd.read_csv('data/communes_code.csv', sep=';',encoding="utf-8")

# Left_join des codes_communes avec les noms des communes geoJson
df = df.merge(commune, left_on=["code_commune"], right_on=['code'], how='left')
df = df.sort_values(by='code_commune', ascending=True)

On souhaite compléter le maximum des transactions qui n'ont pas été géolocalisées précisément dans leur commune.

In [35]:
# Nombre total de transactions renseignées (celles qui ont des valeurs dans 'latitude' et 'longitude')
total_transactions_renseignees = df.shape[0]

# Nombre de lignes où 'latitude' ou 'longitude' est manquant (sans double compte)
non_geolocalisees = df[df['latitude'].isna() | df['longitude'].isna()].shape[0]

# Calcul du pourcentage de transactions non géolocalisées
pourcentage_non_geolocalisees = (non_geolocalisees / total_transactions_renseignees) * 100

# Affichage de la phrase
print(f"Sur les {total_transactions_renseignees} transactions renseignées, {pourcentage_non_geolocalisees:.2f}% ne sont pas géolocalisées.")

Sur les 125887 transactions renseignées, 0.42% ne sont pas géolocalisées.


In [36]:
# Calculer le nombre de transactions par commune
nombre_transactions_cotieres = df.groupby('communeName').size().reset_index(name='Nombre de transactions')

In [37]:
df_missing = df[df['latitude'].isna()]

In [38]:
print(df_missing.value_counts()/df.value_counts*100)

Series([], Name: count, dtype: object)


In [39]:
df_missing = df_missing.drop(columns=['latitude', 'longitude'])

In [40]:
df_missing = geolocaliser.geolocaliser_actifs(df_missing, 'adresse_py', 'latitude', 'longitude')

Erreur de requête pour l'adresse : 19+Rue+DE+L+OCEAN+29840+Porspoder, statut : 429
Erreur de requête pour l'adresse : 2+Rue+DE+CREAC+H+AL+LAN+29000+Quimper, statut : 429
Erreur de requête pour l'adresse : 7+Avenue+DE+KERDREZEC+29000+Quimper, statut : 429
Erreur de requête pour l'adresse : 21+Chemin+DE+KERNOTER+29000+Quimper, statut : 429
Erreur de requête pour l'adresse : +Résidence+DU+STILETTO+20167+Ajaccio, statut : 429
Erreur de requête pour l'adresse : 2+Allée+DE+PEN+RUIC+29000+Quimper, statut : 429
Erreur de requête pour l'adresse : +Résidence+DU+STILETTO+20167+Ajaccio, statut : 429
Erreur de requête pour l'adresse : +Résidence+DU+STILETTO+20167+Ajaccio, statut : 429
Erreur de requête pour l'adresse : 9001+Résidence+COLLINES+DU+SALARIO+20000+Ajaccio, statut : 429
Erreur de requête pour l'adresse : +Résidence+DU+STILETTO+20167+Ajaccio, statut : 429
Erreur de requête pour l'adresse : 46+Résidence+LES+JARDINS+D+APPOLONIE+20200+Bastia, statut : 429
Erreur de requête pour l'adresse : 4

In [41]:
df_missing = df_missing.dropna(subset=['latitude'])

In [42]:
# Concatène les DataFrames en réinitialisant l'index
df_final = pd.concat([df, df_missing], ignore_index=True)

In [43]:
base_transactions = df_final.drop(columns=['departmentCode','communeName','code'])

In [ ]:
base_transactions.to_parquet("data/base.parquet", index=False, engine="pyarrow")

In [44]:
# base_transactions.to_csv("data/base.csv",sep=";",index=False,encoding="utf-8")